In [15]:
from gan_layers import gan_network
import os
import numpy as np
from PIL import Image
import random
import cv2

In [68]:
input_dir = 'train/240p/'
tgt_dir = 'train/720p/'
img_list = os.listdir(input_dir)
assert img_list == os.listdir(tgt_dir)

input_path = input_dir + img_list[0]
input_shape_temp = np.array(Image.open(input_path)).shape

tgt_path = tgt_dir + img_list[0]
tgt_shape_temp = np.array(Image.open(tgt_path)).shape

In [66]:
tgt_shape

(720, 1280, 3)

In [64]:
class ImageDataGenerator(object):
    def __init__(self, input_dir, tgt_dir):
        self.input_dir = input_dir
        self.tgt_dir = tgt_dir
        self.img_list = os.listdir(input_dir)
        assert self.img_list == os.listdir(tgt_dir)
        self.reset()

    def reset(self):
        self.inputs = []
        self.targets = []

    def flow_from_directory(self, batch_size=16):
        train_imges = random.sample(self.img_list, batch_size)
        while True:
            for img in train_imges:
                input_path = input_dir + img
                tgt_path = tgt_dir + img

                X = (np.array(Image.open(input_path)) - 127.5) / 127.5
                Y = (np.array(Image.open(tgt_path)) - 127.5) / 127.5

                X_list.append(X.reshape([1, X.shape[0], X.shape[1], X.shape[2]]))
                Y_list.append(Y.reshape([1, Y.shape[0], Y.shape[1], Y.shape[2]]))
    
            inputs = np.vstack(X_list)
            targets = np.vstack(Y_list)

            yield inputs, targets

In [69]:
input_shape = (None, input_shape_temp)
input_shape

(None, (240, 426, 3))

In [ ]:
input_shape = (None, input_shape_temp)
tgt_shape = (None, tgt_shape_temp)


inputs = Input(shape=noise_shape)
tensors = [inputs]
self.gen_net = []

self.gen_net.append(BatchNormalization(momentum=0.8))
self.gen_net.append(UpSampling2D())
self.gen_net.append(Conv2D(128, kernel_size=3, padding="same"))
self.gen_net.append(Activation("relu"))
self.gen_net.append(BatchNormalization(momentum=0.8))
self.gen_net.append(UpSampling2D())
self.gen_net.append(Conv2D(64, kernel_size=3, padding="same"))
self.gen_net.append(Activation("relu"))
self.gen_net.append(BatchNormalization(momentum=0.8))
self.gen_net.append(Conv2D(3, kernel_size=3, padding="same"))

for i, l in enumerate(self.gen_net):
    tensors.append(l(tensors[i]))


return Model(tensors[0], tensors[-1])

In [ ]:
# 